In [1]:
import ast
import glob
import numpy as np
import pandas as pd
from coverage_functions import *

# ModBase
## Filtering data according to reference proteome & model quality

In [3]:
# reviewed homo sapiens proteome from UniProt 2022_04
# this dataset is filtered according to certain thresholds
proteome = pd.read_excel("processed_data/uniprot/30aa_nounchar_noputative_ref_proteome_protein_existence_filtered_02.xlsx", header=0)
len(proteome)

18401

In [4]:
# ModBase file is too big to load into memory (108070195 lines). Chunking first.
# i=0
# for df in pd.read_csv("raw_data/modbase/modbase_models_all-latest", chunksize=5000000):
#     df.to_csv('raw_data/modbase/modbase_models_all-latest_chunked/file_{:02d}.xlsx'.format(i), index=False)
#     i += 1

In [4]:
# filter wrt proteome
def compare_with_uniprot(file):
    file = file.assign(in_uniprot=file.UniprotID.isin(proteome.Entry))
    return file[file.in_uniprot == True]

# filter by quality and length
def filter_modbase(file):
    # >= 30 aa models
    file = file[file['TargetEnd'] - file['TargetBeg'] + 1 >= 30]
    
    # get high quality models
    # quality thresholds are taken from Interactome3D paper
    # if available, filter according to MPQS.
    file1 = file[file["ModPipeQualityScore"] >= 1.1] 
    
    # if MPQS is not available, filter wrt Sequence Identity AND Model Score
    file2 = file[file.ModPipeQualityScore.isnull()] 
    file2 = file2[(file2["SeqIdentity"] >= 30) & (file2["ModelScore"] >= 0.7)] 
    file3 = pd.merge(file1, file2, how="outer")
    
    # duplicate filtering, keep the highest scores. 
    filtered_file = file3.sort_values(by=['ModPipeQualityScore', 'SeqIdentity' , 'ModelScore']).drop_duplicates(subset=["UniprotID", "TargetBeg", "TargetEnd"], keep="last")

    return filtered_file

In [18]:
# read all files and filter wrt proteome
modbase_files = glob.glob("raw_data/modbase/modbase_models_all-latest_chunked/*.xlsx")

for f in modbase_files:
    file = pd.read_csv(f, delimiter = "\t", dtype={'PDBBegin': 'object','PDBEnd': 'object'}) # for unicode errors
    in_proteome = compare_with_uniprot(file)
    in_proteome.to_csv("processed_data/modbase/filtered_proteome/modbase_" + f.split("/")[-1].split(".")[0] + ".tsv", sep="\t", index=False)

In [5]:
# read all files and concatanate
folder = 'processed_data/modbase/filtered_proteome'
modbase_proteome = pd.concat([pd.read_csv(f, sep='\t', dtype={'PDBBegin': 'object','PDBEnd': 'object'}) for f in glob.glob(folder + "/*.tsv")], ignore_index=True)
print(len(modbase_proteome)) # 400096

400096


In [6]:
modbase_30aa_hq = filter_modbase(modbase_proteome) # 35921
modbase_30aa_hq.to_csv("processed_data/modbase/modbase_30aa_hq.tsv", sep="\t", index=False)
len(modbase_30aa_hq)

35921

## Coverage Calculation & Correction

Coverage of some proteins are higher than 100. Lengths of the proteins are incorrect in ModBase, resulting in incorrect residue ranges in models. We need to delete those models.

### Coverage Calc

In [ ]:
# we will work on the filtered file
modbase_30aa_hq = pd.read_csv("processed_data/modbase/modbase_30aa_hq_fixed.tsv", sep="\t") 

In [12]:
modbase_30aa_hq = modbase_30aa_hq.rename(columns={'UniprotID': 'Entry'})
modbase_30aa_hq['combined'] = modbase_30aa_hq.apply(lambda x: list([x["TargetBeg"], x["TargetEnd"]]),axis=1)
data_new1 = modbase_30aa_hq.groupby('Entry').combined.apply(list).reset_index() 
data_new2 = modbase_30aa_hq.groupby(['Entry'], as_index=False).agg({"Length":"first"}) 
modbase_30aa_hq_cov = pd.merge(data_new1, data_new2, on="Entry")
modbase_30aa_hq_cov # 8618 proteins

,Entry,combined,Length
0,A0A578,"[[16.0, 113.0], [15.0, 113.0], [22.0, 114.0]]",114
1,A0A584,"[[20.0, 115.0], [22.0, 115.0]]",115
2,A0A5B0,"[[20.0, 111.0]]",115
3,A0A5B6,"[[15.0, 113.0]]",114
4,A0AV96,"[[143.0, 245.0], [147.0, 245.0]]",593
...,...,...,...
8613,Q9Y6X8,"[[518.0, 605.0], [524.0, 605.0], [444.0, 496.0...",837
8614,Q9Y6Y0,"[[5.0, 127.0]]",642
8615,Q9Y6Y1,"[[872.0, 955.0], [872.0, 954.0]]",1673
8616,Q9Y6Y9,"[[17.0, 160.0]]",160


In [13]:
# calculate_coverage function comes from coverage_functions.py 
modbase_30aa_hq_cov["Coverage"] = modbase_30aa_hq_cov.apply(lambda x: calculate_coverage(begin_end = x["combined"],
                                                                                         prot_length = x["Length"]), axis=1)

### Correction part

In [16]:
# models with correct length
correct_modbase_30aa_hq_cov = modbase_30aa_hq_cov[~(modbase_30aa_hq_cov.Coverage > 100)] # 8618 # 22 proteins have wrong length info
correct_modbase_30aa_hq_cov.to_excel('processed_data/modbase/modbase_30aa_hq_fixed_cov.xlsx', index=False)
len(correct_modbase_30aa_hq_cov)

8618

In [56]:
correct_modbase_30aa_hq_cov = pd.read_excel('processed_data/modbase/modbase_30aa_hq_cov.xlsx')
correct_modbase_30aa_hq_cov.combined.apply(lambda x: ast.literal_eval(x)).apply(len).sum() # 35808

35808

In [57]:
modbase_30aa_hq_fixed = modbase_30aa_hq[modbase_30aa_hq.Entry.isin(correct_modbase_30aa_hq_cov.Entry)]
modbase_30aa_hq_fixed.to_csv("processed_data/modbase/modbase_30aa_hq_fixed.tsv", sep="\t", index=False)
len(modbase_30aa_hq_fixed) # 35808 models, correct!

35808

In [ ]:
# triple checking
mod_res_cov = get_residue_coverage(modbase_30aa_hq_fixed)
len(mod_res_cov)